# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [2]:
import pandas as pd
import requests
# Configurar para exibir mais linhas e colunas
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

# Configurar para mostrar todo o conteúdo das colunas
pd.set_option('display.max_colwidth', None)

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [3]:
# 1) Seu código aqui
url = 'http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Município'

# Fazendo a requisição HTTP GET
response = requests.get(url)

# Verificando se a requisição foi bem-sucedida
if response.status_code == 200:
    # Convertendo a resposta JSON para um dicionário Python
    data = response.json()
    
    # Convertendo o dicionário para um DataFrame do Pandas
    # A chave 'items' contém a lista de registros
    df = pd.json_normalize(data['items'])
    
    # Exibindo as primeiras linhas do DataFrame para verificar os dados carregados
    print(df.head(10))
else:
    print(f'Falha na requisição. Status code: {response.status_code}')

   id_pleito tipo_interessado            interessado  cod_ibge  uf  \
0      66751        Município               Mesquita   3302858  RJ   
1      61009        Município         Rio das Flores   3304508  RJ   
2      50533        Município            Nova Iguaçu   3303500  RJ   
3       9205        Município  Campos dos Goytacazes   3301009  RJ   
4       7986        Município              Rio Claro   3304409  RJ   
5      12521        Município            Nova Iguaçu   3303500  RJ   
6       8839        Município            Porciúncula   3304102  RJ   
7      36529        Município                Resende   3304201  RJ   
8      34869        Município            Nova Iguaçu   3303500  RJ   
9      65407        Município               Itaboraí   3301900  RJ   

                num_pvl                          status          num_processo  \
0  PVL02.003123/2023-87                        Deferido  17944.104458/2023-84   
1  PVL02.007214/2022-19  Arquivado por decurso de prazo  17944.1030

In [4]:
# 2) Seu código aqui
df['status'].value_counts()

status
Deferido                                                 106
Arquivado                                                 53
Regularizado                                              26
Arquivado por decurso de prazo                            24
Deferido (PVL-IF)                                         24
Encaminhado à PGFN com manifestação técnica favorável     14
Arquivado a pedido                                         8
Em retificação pelo interessado                            3
Em análise                                                 2
Arquivado a pedido (PVL-IF)                                1
Arquivado pela STN                                         1
Name: count, dtype: int64

In [5]:
# 3) Seu código aqui 
# df['ano_status'] = df['data_status'].dt.year
print(df['data_status'].dtype)
df['ano_status'] = df['data_status'].astype(str).str[-4:]
df['ano_status'].head(10)

object


0    2016
1    2012
2    2006
3    2012
4    2013
5    2009
6    2016
7    2011
8    2013
9    2013
Name: ano_status, dtype: object

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [9]:
#1) Seu código aqui
def baixar_dataframe_uf(uf, tipo_interessado):
    url = f'http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf={uf}&tipo_interessado={tipo_interessado}'
    response = requests.get(url)# Verificando se a requisição foi bem-sucedida
    
    if response.status_code == 200:
        # Convertendo a resposta JSON para um dicionário Python
        data = response.json()
        
        # Convertendo o dicionário para um DataFrame do Pandas
        # A chave 'items' contém a lista de registros
        df = pd.json_normalize(data['items'])
        
        # Exibindo as primeiras linhas do DataFrame para verificar os dados carregados
        df.head(10)
        return df
    else:
        print(f'Falha na requisição. Status code: {response.status_code}')


In [36]:
# 2) Seu código aqui
df_mg = baixar_dataframe_uf('MG','Estado')
df_mg.head(10)

# separo um DF com os deferidos
df_mg_deferido = df_mg[df_mg['status'] == 'Arquivado por decurso de prazo']

print(f'Foi encontrado {df_mg_deferido.shape[0]} registro com status em "Arquivado por decurso de prazo"')

Foi encontrado 1 registro com status em "Arquivado por decurso de prazo"


In [29]:
# 3) Seu código aqui
df_ba = baixar_dataframe_uf('BA','Município')

# separo um DF com os deferidos
df_ba_deferido = df_ba[df_ba['status'] == 'Deferido']
# faço a contagem de municípios ("interessado") com status deferido
contagem_interessado = df_ba_deferido['interessado'].value_counts()

print(contagem_interessado)

interessado
Luís Eduardo Magalhães       16
Vitória da Conquista         12
Lauro de Freitas             11
Barreiras                    10
Camaçari                     10
                             ..
Barra do Choça                1
Caturama                      1
Presidente Tancredo Neves     1
Amélia Rodrigues              1
Paulo Afonso                  1
Name: count, Length: 185, dtype: int64


In [37]:
# 4) Seu código aqui
df_ba2 = baixar_dataframe_uf('BA','Estado')
df_ba2.to_csv('dados_ba.csv', index=False)